## Levatamos Fermionic

In [ ]:
import numpy as np
import openfermion as of
from tqdm import tqdm
from itertools import combinations
from openfermion.utils import commutator, count_qubits, hermitian_conjugated
import functools
import concurrent.futures
from numba import njit
import tensorflow as tf
import scipy
import sparse
import itertools
import linecache
from multiprocessing import Pool, cpu_count

# Generación de base
class fixed_basis:
    @staticmethod
    def int_to_bin(k, d):
        return np.base_repr(k, 2).zfill(d)

    @staticmethod
    def bin_to_op(b):
        tups = [(i, 1) for i, k in list(enumerate(list(b))) if k == '1']
        return of.FermionOperator(tups)

    def idx_to_repr(self, idx):
        return self.canonicals[idx]

    def opr_to_idx(self, opr):
        for i in range(self.size): # Evitar esto ordenando opr
            if self.base[i] == opr:
                return i

    # Calcula el valor medio a partir del indice del vector y el operador
    def idx_mean_val(self, idx: int, op: of.FermionOperator):
        vec = self.idx_to_repr(idx)
        return np.real(np.transpose(vec) @ of.get_sparse_operator(op, n_qubits=self.d) @ vec)

    # Calcula el valor medio a partir de un estado y el operador
    def mean_val(self, vec, op):
        idx = self.opr_to_idx(vec)
        return self.idx_mean_val(idx, op)

    # Calcula la contracción de un operador sobre dos estados dados
    def idx_contraction(self, idx_1, idx_2, op):
        rep = lambda x: self.idx_to_repr(x)
        return np.real(np.transpose(rep(idx_1)) @ of.get_sparse_operator(op, n_qubits=self.d) @ rep(idx_2))

    def create_basis(self, d, num = None, pairs = False):
        basis = []
        num_ele = []
        for k in range(0,2**d):
            b = self.int_to_bin(k, d)
            if num != None:
                if b.count('1') == num:
                    if pairs:
                        if np.all(b[::2] == b[1::2]):
                            oper = self.bin_to_op(b)
                            basis.append(oper)
                            num_ele.append(k)
                    else:
                        oper = self.bin_to_op(b)
                        basis.append(oper)
                        num_ele.append(k)
            else:
                oper = self.bin_to_op(b)
                basis.append(oper)
        return basis, num_ele

    def __init__(self, d, num = None, pairs = False, basis = None, num_ele = None):
        self.d = d
        self.num = num
        self.m = num
        # Si nos da la base, la levantamos (asumimos GC). Si no, la creamos
        if basis is None:
            self.base, self.num_ele = self.create_basis(d, num, pairs)
        else:
            self.base, self.num_ele = basis, num_ele
        self.size = len(self.base)
        self.canonicals = np.eye(self.size)
        self.pairs = pairs

    @staticmethod
    def cdc(i, j):
        return of.FermionOperator(((i,1),(j,0)))

    @staticmethod
    def cc(i, j):
        return of.FermionOperator(((i,0),(j,0)))

    # Del indice, cuenta el número de partículas
    def num_idx(self, idx):
        b = self.int_to_bin(idx, basis.d)
        return b.count('1')

    # Calculo de rho1 (via directa, lento, y solo definido en la base por ahora)
    def rho_1(self, op):
        # Necesitamos un índice, es?
        if type(op) != int:
            op = self.opr_to_idx(op)
        mat = np.zeros((self.d, self.d))
        for i in range(self.d):
            for j in range(self.d):
                cdc = self.cdc(j, i)
                mat[i,j] = self.idx_mean_val(op, cdc)
        return mat

# Calculo de generadores de rho1
def rho_1_gen(basis):
    # Vamos a crear un hipersparse de TF, almacenamos los valores acá
    indices = []
    values = []
    shape = (basis.d, basis.d, basis.size, basis.size)
    d = basis.d
    for i in tqdm(range(0, d)):
        for j in range(0, d):
            # Generamos el operador
            op = basis.cdc(j, i)
            #print(op)
            if basis.num == None:
                mat = np.real(of.get_sparse_operator(op, n_qubits=d))
            else:
                mat = np.real(of.get_sparse_operator(op, n_qubits=d))[np.ix_(basis.num_ele, basis.num_ele)]
            # Extraemos la información
            n_r, n_c = mat.nonzero()
            data = mat.data
            for r, c, v in zip(n_r, n_c, data):
                indices.append([i, j, r, c])
                values.append(v)
    indices_t = np.array(indices).T
    s_t = sparse.COO(indices_t, values, shape=shape)
    return s_t

# Calculo de rho1 (via generadores) de un vector en la base canonica
def rho_1(vect, rho_1_arrays):
    if len(vect.shape) == 1: # vectores
        return sparse.einsum('k,ijkl,l->ij', vect, rho_1_arrays, vect)
    elif len(vect.shape) == 2: # mat densidad
        return sparse.einsum('ijkl,kl->ij', rho_1_arrays, vect)
    else: # mat densidad batcheadas
        return sparse.einsum('bkl,ijkl->bij', vect, rho_1_arrays)

# Calculo de indices de rho2kkbar
def get_kkbar_indices(t_basis):
    indices = []
    for i, ind in enumerate(t_basis.num_ele):
        v = t_basis.int_to_bin(ind, t_basis.d)
        if np.all(v[::2] == v[1::2]):
            indices.append(i)
    return indices

# Calculo de generadores de rho2
def rho_2_gen(basis, t_basis, idx_list = []):
    # Vamos a crear un hipersparse de TF, almacenamos los valores acá
    d = basis.d
    indices = []
    values = []
    if len(idx_list) == basis.m:
        idx_list = idx_list
    elif len(idx_list) == basis.m**4:
        idx_list = np.unique(idx_list[:,0])
    else:
        idx_list = range(t_basis.size)
    shape = (len(idx_list), len(idx_list), basis.size, basis.size)
    for i, ii in tqdm(enumerate(idx_list), total=len(idx_list)):
        for j, jj in enumerate(idx_list):
            # Generamos el operador
            op = t_basis.base[jj]*of.utils.hermitian_conjugated(t_basis.base[ii])
            if basis.num == None:
                mat = np.real(of.get_sparse_operator(op, n_qubits=d))
            else:
                mat = np.real(of.get_sparse_operator(op, n_qubits=d))[np.ix_(basis.num_ele, basis.num_ele)]
            # Extraemos la información
            n_r, n_c = mat.nonzero()
            data = mat.data
            for r, c, v in zip(n_r, n_c, data):
                indices.append([i, j, r, c])
                values.append(v)

    indices_t = np.array(indices).T
    s_t = sparse.COO(indices_t, values, shape=shape)
    return s_t

# rho_m_gen aux func
def process_chunk(args):
    chunk, m_basis, basis, d, it_set = args
    indices = []
    values = []
    for ii in chunk:
        for jj in it_set:
            # Generate the operator
            op = m_basis.base[jj] * of.utils.hermitian_conjugated(m_basis.base[ii])
            mat = np.real(of.get_sparse_operator(op, n_qubits=d))[np.ix_(basis.num_ele, basis.num_ele)]
            # Extract the information
            n_r, n_c = mat.nonzero()
            data = mat.data
            for r, c, v in zip(n_r, n_c, data):
                indices.append([ii, jj, r, c])
                values.append(v)
    return indices, values

# Parallelized rho_m_gen
def rho_m_gen(basis, m, num_workers=None):
    if num_workers is None:
        num_workers = cpu_count()  # Use all available CPUs by default
    
    indices = []
    values = []
    m_basis = fixed_basis(basis.d, num=m, pairs=basis.pairs)
    shape = (m_basis.size, m_basis.size, basis.size, basis.size)

    it_set = np.arange(m_basis.size)
    chunks = np.array_split(it_set, num_workers)  # Split `it_set` into chunks for each worker

    # Use multiprocessing Pool for parallel processing
    with Pool(processes=num_workers) as pool:
        # Pass arguments as tuples instead of using a lambda
        results = list(
            tqdm(
                pool.imap(
                    process_chunk, 
                    [(chunk, m_basis, basis, basis.d, it_set) for chunk in chunks]
                ),
                total=num_workers
            )
        )
    
    # Collect results from all processes
    for indices_chunk, values_chunk in results:
        indices.extend(indices_chunk)
        values.extend(values_chunk)

    # Construct the sparse array
    indices_t = np.array(indices).T
    s_t = sparse.COO(indices_t, values, shape=shape)
    return s_t

def rho_m(vect, rho_m_arrays):
    return sparse.einsum('k,ijkl,l->ij', vect, rho_m_arrays, vect)

# Calculo de rho2 (via generadores) de un estado en la base canonica
def rho_2(vect, rho_2_arrays):
    if len(vect.shape) == 1: # vectores SOLO RHO2 COMPLETA
        return sparse.einsum('k,ijkl,l->ij', vect, rho_2_arrays, vect)
    elif len(vect.shape) == 2: # mat densidad SOLO RHO2 COMPLETA
        return sparse.einsum('ijkl,kl->ij', rho_2_arrays, vect)
    else: # mat densidad batcheadas
        return sparse.einsum('bkl,ijkl->bij', vect, rho_2_arrays)

# Calculo de generadores de K (usado para quasiparticles) WIP SPARSE
def k_gen(basis):
    mat = np.zeros((basis.d, basis.d, basis.size, basis.size))
    d = basis.d
    for i in tqdm(range(0, d), total=d):
        for j in range(0, d):
            op = basis.cc(j, i)
            if basis.num == None:
                mat[i,j,::] = np.real(of.get_sparse_operator(op, n_qubits=d)).todense()
            else:
                mat[i,j,::] = np.real(of.get_sparse_operator(op, n_qubits=d)).todense()[np.ix_(basis.num_ele, basis.num_ele)]
    return mat

def k_vect(vect, k_gen):
    return np.einsum('k,ijkl,l->ij', vect, k_gen, vect)

# Calculo la matrix rho de cuasipartículas  WIP SPARSE
def rho_qsp(vect, rho_1_arrays, k_arrays, rho1 = None):
    if type(rho1) == None:
        rho1 = rho_1(vect, rho_1_arrays)
    k = k_vect(vect, k_arrays)

    mat = np.block([[rho1, k], [-np.conjugate(k), np.eye(rho_1_arrays.shape[0])-np.conjugate(rho1)]])
    return mat

# Devuelve los indices que tienen a level ocupado
def level_proy(d, level):
    ids = []
    for k in range(0,2**d):
        b = fixed_basis.int_to_bin(k, d)
        if b[level] == '1':
            ids.append(k)
    arr = np.zeros(2**d)
    arr[np.array(ids)] = 1
    return arr, ids

def parity_levels(d):
    rng = range(2**d)
    binary_repr = np.vectorize(np.binary_repr)(rng)
    ones_c = np.char.count(binary_repr, '1')
    return np.array(rng)[ones_c % 2 == 1] # seleccionamos estados impares

# Devuelve el vector postmedido
def measure(basis, vect, level = 1):
    l_arr, l_ids = level_proy(basis.d, level)
    proy_v = vect * l_arr
    comp_arr = np.logical_not(l_arr).astype(int)
    comp_v = vect * comp_arr
    norm = lambda v: v / np.linalg.norm(v)
    return norm(proy_v), norm(comp_v)

def entropy(rho, m):
    S_fun = lambda rho: -1*np.trace(rho @ scipy.linalg.logm(rho)) / np.log(2)
    ent = S_fun(rho) / (np.log2(scipy.special.binom(basis.d, m)))
    return ent

# Levanta bases de QChem
def build_csv_basis(csvf, d):
    # Construimos la base
    ops = []
    with open(csvf, 'r') as basis:
        num_ele = []
        # Contar los niveles
        m_level = 0
        # Creamos operadores
        for l in basis.read().splitlines()[4:]:
            natop = [int(x) for x in l.split(' ')[1:]] # Operador en forma de lista
            #print(natop)
            op = of.FermionOperator(([(i, 1) for i in natop]))
            ops.append(op)
            # Contamos niveles
            m_level = max(m_level, *natop)
            # Determinamos el índice
            natop_to_int = lambda x: np.sum([2**(d-1-i) for i in x])
            num_ele.append(natop_to_int(natop))
 
        # Determinamos m y d
        num = len(natop)
        #assert d == m_level+1

    return fixed_basis(d, num = num, pairs = False, basis = ops, num_ele=num_ele)

2025-01-17 14:40:38.727409: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-17 14:40:38.727432: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-17 14:40:38.728287: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-17 14:40:38.733194: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-17 14:40:39.415648: W tensorflow/compiler/tf2

Debido al número de niveles (innecesarios) en el core, el objetivo es calcular la base natural (a partir de rho1), y proyectar el fundamental sobre el subespacio generado por los estados no ocupados. Allí, buscamos la descomposición, y luego volvemos a la base original

In [24]:
# Aux func
def remove_null_terms(op):
    op_nn = of.FermionOperator.zero()
    terms = list(op.terms.items())
    act_idx = lambda tt: np.array([i[0] for i in tt[0]])
    filtered_terms = []

    for term in terms:
        acts = np.array(act_idx(term))
        u, c = np.unique(acts, return_counts=True)
        if np.max(c) == 1:
            filtered_terms.append(term)

    return filtered_terms

def op_to_rep(basis, op): # CHEQUEADO
    op = -1*of.transforms.normal_ordered(op)
    terms = list(op.terms.items())
    act_idx = lambda tt: [i[0] for i in tt[0]]
    vect = np.zeros(basis.size)

    for term in terms:
        act = act_idx(term)
        nele = np.sum([2**(basis.d-1-i) for i in act]) # si
        nele_idx = np.argwhere(basis.num_ele == nele)[0][0]
        vect[nele_idx] += term[1]
    
    return vect
    
def build_basis_from_vect(basis, vect, tol = 1e-10):
        ops = []
        num_ele = []
        vect_pro = []
        for idx, coord in enumerate(vect):
                if np.abs(coord) > tol:
                        ops.append(basis.base[idx])
                        num_ele.append(basis.num_ele[idx])
                        vect_pro.append(vect[idx])
        return fixed_basis(basis.d, num = basis.num, pairs = False, basis = ops, num_ele=num_ele), np.array(vect_pro)


In [4]:
# Dado la base, una rotación entre los 1SP states y un vector (representado en basis)
# devuelve la representación en la base natural
def natural_basis_rotation(basis, C, vect):
    # Calculamos la transformación de cada generador
    targ_sp = []
    for i in range(basis.d):
        op = np.sum([of.FermionOperator(((basis.d-1-j,1)),C[i,j]) for j in range(basis.d) if np.abs(C[i,j]) > 1e-5])
        targ_sp.append(op)

    targ_sp.reverse()

    # Reemplazamos en el fundamental
    vect_op = np.sum([vect[i] * basis.base[i] for i in range(len(vect))])
    terms = list(vect_op.terms.items())

    vect_op_t = of.FermionOperator.zero()
    for term in terms:
        # Construimos el término
        act_idx = lambda tt: [i[0] for i in tt[0]]
        tt = np.prod([targ_sp[i] for i in act_idx(term)])
        vect_op_t += term[1] * tt
        
    # Filtramos los términos nulos
    f_terms = remove_null_terms(vect_op_t)
    vect_op_ft = of.FermionOperator.zero()
    for ft in f_terms:
        vect_op_ft += of.FermionOperator(ft[0],ft[1])

    # Calculamos la representación en la base
    ext_basis = fixed_basis(basis.d, basis.m)
    res = op_to_rep(ext_basis, vect_op_ft)
    res = 1/np.linalg.norm(res) * res

    return res, vect_op_ft

### Determinación de estado

##### Cargamos el input

In [9]:
# Levantamos la base, y calculamos rho1
basis = build_csv_basis('hf_basis', 12)
rho_1_arrays = rho_m_gen(basis, 1, num_workers=32)

# Levantamos el fundanental d=3
vect = [7.211630245189536e-01,4.206455224775346e-02,-2.285388031865260e-12,1.699211387138645e-12,-4.129479393831875e-03,-3.823465330043704e-07,4.206455225178431e-02,-6.901551297130625e-01,2.105502564059137e-12,-1.565639815440850e-12,-4.122956845587538e-03,-3.755879654736101e-05,-7.228479653547532e-12,6.636332640706462e-12,-9.140952135390417e-04,-4.046086775026300e-16,8.239170289440526e-14,2.416202424789693e-16,-8.029397050081800e-13,7.469929098902930e-13,-1.075293384255381e-14,-9.140952135362526e-04,4.469835410063215e-15,5.163385528935227e-17,-4.129479393815261e-03,-4.122956845567992e-03,3.604262375317366e-14,-2.262779577530237e-14,-1.075992639441983e-03,-2.588630943109245e-05,-3.823465329666192e-07,-3.755879654741322e-05,5.671459804258443e-17,-2.094441474950918e-16,-2.588630943109360e-05,-8.771828180833955e-06]
vect = np.array(vect)

# Rho1 + Rho2
rho_1_obj  = np.sort(np.concatenate([np.repeat(1/2, 2*2), np.repeat(1, 4*2)]))
rho_2_obj = np.sort(np.concatenate([np.repeat(1/2, 8*2+16), np.repeat(1, 6*2+17), np.repeat(0, 5)]))

100%|██████████| 32/32 [00:01<00:00, 25.82it/s]


In [27]:
# Cambio a base natural 
rho_1 = rho_m(vect, rho_1_arrays).todense()
evals, evects = scipy.linalg.eigh(rho_1)
C = evects
vect_pro, op_pro = natural_basis_rotation(basis, C, vect)
# Calculamos la base extendida, donde vive vect_pro y restringimos
ext_basis = fixed_basis(basis.d, basis.m)
basis_pro, vect_pro = build_basis_from_vect(ext_basis, vect_pro)
# Calculamos los arrays en esta base
rho_1_arrays = rho_m_gen(basis_pro, 1, num_workers=32)
rho_2_arrays = rho_m_gen(basis_pro, 2, num_workers=32)

100%|██████████| 32/32 [00:36<00:00,  1.15s/it]


In [36]:
from scipy.cluster.hierarchy import fcluster, linkage
from scipy.cluster.vq import kmeans, vq

# Input

# Aux functions
r_eig = lambda x: np.sort(np.real(np.linalg.eigvals(x.todense())))
k = 2 # Número de clusters

def sparsity_error(A, k=2):
    # L1
    sparsity_term = np.sum(np.abs(A))

    # Clustering 
    non_zero_coeffs = A[np.abs(A) > 1e-5]  # Sacamos los ceros
    if len(non_zero_coeffs) > 0:
        # Calculamos los clusters (k) y la distancia en ellos
        centroids, _ = kmeans(non_zero_coeffs, k)
        cluster_labels, distances = vq(non_zero_coeffs, centroids)
        cluster_error = np.sum(distances**2) 
    else:
        cluster_error = 0

    loss = 1 * sparsity_term + 2 * cluster_error
    return loss

# Loss definition
def loss_fun(vect):
    # Rho 2 loss
    rho = rho_m(vect, rho_2_arrays)
    eigv = r_eig(rho)
    lrho2 = np.linalg.norm(rho_2_obj-eigv)
    # Rho 1 loss
    rho = rho_m(vect, rho_1_arrays)
    eigv = r_eig(rho)
    lrho1 = np.linalg.norm(rho_1_obj-eigv)
    # L1 + clustering loss
    l1 = sparsity_error(vect, k) 

    #print(lrho2, lrho1, l1)
    return lrho2 + lrho1 + 1/2*l1

opt = scipy.optimize.minimize(loss_fun, vect_pro, method='L-BFGS-B')

In [42]:
print(opt)
print(opt.x)
print(sparsity_error(opt.x, k=k))
print(rho_1_obj)
np.round(r_eig(rho_m(opt.x, rho_1_arrays)),2)

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 1.1789452999374304
        x: [-1.115e-03  1.208e-03 ...  1.187e-03 -7.072e-01]
      nit: 11
      jac: [-1.195e+00  3.999e-01 ...  2.300e-01 -5.037e+00]
     nfev: 1232
     njev: 77
 hess_inv: <15x15 LbfgsInvHessProduct with dtype=float64>
[-1.11479488e-03  1.20847971e-03 -1.12196875e-03  1.20837150e-03
 -9.99996232e-06  1.62021695e-03 -2.74187038e-04  1.00462928e-03
  8.34930618e-04  1.20963641e-03 -1.18130874e-03  7.07007588e-01
 -1.20426990e-03  1.18745490e-03 -7.07228056e-01]
2.349528518023165
[0.5 0.5 0.5 0.5 1.  1.  1.  1.  1.  1.  1.  1. ]


array([0.5, 0.5, 0.5, 0.5, 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ])

In [44]:
final_res = of.FermionOperator.zero()
tol = 1e-2
for idx, coord in enumerate(opt.x):
    final_res += basis_pro.base[idx] * coord if np.abs(coord) > tol else 0

final_res


-0.7072280557168331 [0^ 1^ 2^ 3^ 4^ 5^ 6^ 7^ 8^ 9^] +
0.7070075877513028 [0^ 1^ 2^ 3^ 4^ 5^ 6^ 7^ 10^ 11^]

#### Testing

Verificamos que los eig de rho1 se preservan

In [ ]:
# Cálculo de base natural
rho_1 = rho_m(vect, rho_1_arrays).todense()
evals, evects = scipy.linalg.eigh(rho_1)
C = evects

res, vect_op_ft = natural_basis_rotation(basis, C, vect)

ext_basis = fixed_basis(basis.d, basis.m)
rho_1_arrays_n = rho_m_gen(ext_basis, 1, num_workers=32)
rho_1_n = rho_m(res, rho_1_arrays_n).todense()
print(np.trace(rho_1_n), np.trace(rho_1))

evv = lambda vv: np.sort(np.real(np.linalg.eigvals(vv)))
print(evv(rho_1), evv(rho_1_n))

rep_to_op = lambda vect: np.sum([vect[i] * basis.base[i] for i in range(len(vect))])
#unique_ele(vect_op_ft), unique_ele(rep_to_op(vect))

100%|██████████| 32/32 [00:01<00:00, 26.24it/s]


10.000000000000002 9.999999999999996
[0.47807955 0.47807955 0.52192329 0.52192329 0.99999884 0.99999884
 0.99999916 0.99999916 0.99999916 0.99999916 1.         1.        ] [0.47807947 0.47807947 0.52192336 0.52192336 0.99999884 0.99999884
 0.99999916 0.99999916 0.99999916 0.99999916 1.         1.        ]


Verificamos que la composición, es la identidad

In [ ]:
res, vect_op_ft = natural_basis_rotation(basis, C, vect)
res2, vect_op_ft2 = natural_basis_rotation(ext_basis, C.T, res)

vect_op = np.sum([vect[i] * basis.base[i] for i in range(len(vect))])

of.transforms.normal_ordered(vect_op_ft2), res2, of.transforms.normal_ordered(vect_op)

(-0.7211630244701693 [10^ 9^ 8^ 7^ 6^ 4^ 3^ 2^ 1^ 0^] +
 -0.04206455117791357 [10^ 9^ 8^ 7^ 6^ 5^ 3^ 2^ 1^ 0^] +
 0.0041294638743610106 [10^ 9^ 8^ 7^ 6^ 5^ 4^ 3^ 2^ 0^] +
 1.722640320119456e-06 [10^ 9^ 8^ 7^ 6^ 5^ 4^ 3^ 2^ 1^] +
 -0.04206456374360802 [11^ 9^ 8^ 7^ 6^ 4^ 3^ 2^ 1^ 0^] +
 0.6901550609047076 [11^ 9^ 8^ 7^ 6^ 5^ 3^ 2^ 1^ 0^] +
 0.004122938575493033 [11^ 9^ 8^ 7^ 6^ 5^ 4^ 3^ 2^ 0^] +
 3.7602921241802925e-05 [11^ 9^ 8^ 7^ 6^ 5^ 4^ 3^ 2^ 1^] +
 0.0009140907457695419 [11^ 10^ 8^ 7^ 6^ 5^ 4^ 2^ 1^ 0^] +
 2.9256080028949687e-08 [11^ 10^ 8^ 7^ 6^ 5^ 4^ 3^ 1^ 0^] +
 -2.907592256480154e-08 [11^ 10^ 9^ 7^ 6^ 5^ 4^ 2^ 1^ 0^] +
 0.0009140942661331014 [11^ 10^ 9^ 7^ 6^ 5^ 4^ 3^ 1^ 0^] +
 0.0041294638037233785 [11^ 10^ 9^ 8^ 6^ 4^ 3^ 2^ 1^ 0^] +
 0.004122938790304289 [11^ 10^ 9^ 8^ 6^ 5^ 3^ 2^ 1^ 0^] +
 0.0010758547983938326 [11^ 10^ 9^ 8^ 6^ 5^ 4^ 3^ 2^ 0^] +
 2.5887452734296404e-05 [11^ 10^ 9^ 8^ 6^ 5^ 4^ 3^ 2^ 1^] +
 1.7226422573101622e-06 [11^ 10^ 9^ 8^ 7^ 4^ 3^ 2^ 1^ 0^] +
 3.760292

Funciones auxiliares

In [ ]:
def unique_ele(op):
    terms = list(vect_op.terms.items())
    act_idx = lambda tt: [i[0] for i in tt[0]]
    terms_set = []
    for term in terms:
        terms_set.append(act_idx(term))
    return set(terms_set[0]).intersection(*terms_set[1:])

unique_ele(vect_op_ft), unique_ele(vect_op)

({0, 1, 2, 6}, {0, 1, 2, 6})

In [ ]:
# Verifiquemos que los orbitales son ortogonales
op_arr = np.zeros((basis.d,basis.d),dtype=object)
sp_basis = fixed_basis(basis.d, 1)
for i in tqdm(range(basis.d)):
    for j in range(basis.d):
        oi = np.real(of.get_sparse_operator(targ_sp[i], n_qubits=basis.d))
        oj = np.real(of.get_sparse_operator(targ_sp[j], n_qubits=basis.d))
        # Sin daguear y dagueado
        assert len((oi * oj + oj * oi).data) == 0
        assert len((oi.T * oj.T + oj.T * oi.T).data) == 0
        # Términos cruzados
        op_arr[i,j] = oi * oj.T + oj.T * oi
        sh = op_arr[i,j].shape[0]
        if i != j:
            assert np.allclose(np.zeros((sh,sh)),op_arr[i,j].todense())
        else:
            assert np.allclose(np.eye(sh),op_arr[i,j].todense())

100%|██████████| 12/12 [01:00<00:00,  5.07s/it]


In [32]:
est = 0
app_r = lambda x: np.round(1/(x**2), 0)
for i, ii in enumerate(opt.x):
    if np.abs(ii) > 0.1:
        print(i, ii, basis.base[i])
        if np.abs(ii) > 0.2:
            print(app_r(ii))
            est += 1/np.sqrt(app_r(ii)) * basis.canonicals[i]
            print('')

#(1/np.sqrt(3), 1/np.sqrt(12))
#print(est)
est = 1/np.sqrt(12) * (-basis.canonicals[5]+basis.canonicals[25]+basis.canonicals[85]-basis.canonicals[105])+1/np.sqrt(3)*(basis.canonicals[45]+basis.canonicals[65])

print(r_eig(rho_m(est, rho_2_arrays)))


5 -0.27936976320734497 1.0 [0^ 1^ 2^ 3^ 4^ 7^ 8^ 9^ 12^ 13^]
13.0

25 0.28996316812175205 1.0 [0^ 1^ 2^ 3^ 5^ 7^ 8^ 9^ 11^ 13^]
12.0

45 0.5718530568911167 1.0 [0^ 1^ 2^ 3^ 6^ 7^ 8^ 9^ 11^ 12^]
3.0

65 0.572154480816062 1.0 [0^ 1^ 2^ 4^ 5^ 7^ 8^ 9^ 10^ 13^]
3.0

85 0.2897326805359033 1.0 [0^ 1^ 2^ 4^ 6^ 7^ 8^ 9^ 10^ 12^]
12.0

105 -0.27954194550717076 1.0 [0^ 1^ 2^ 5^ 6^ 7^ 8^ 9^ 10^ 11^]
13.0

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.08333333
 0.08333333 0.08333333 0.08333333 0.08333333 0.08333333 0.08333333
 0.08333333 0.08333333 0.08333333 0.08333333 0.08333333 0.33333333
 0.33333333 0.33333333 0.33333333 0.33333333 0.33333333 0.5
 0.5        0.5        0.5        0.5        0

In [259]:
r_eig(rho_m(vect, rho_2_arrays))

array([6.39649654e-18, 2.27759896e-17, 2.27759896e-17, 2.97340303e-17,
       1.05847838e-08, 1.05847839e-08, 8.12469946e-02, 8.12469946e-02,
       8.12469946e-02, 8.12469946e-02, 8.12469946e-02, 8.12469946e-02,
       8.26670825e-02, 8.26670825e-02, 8.26670825e-02, 8.26670825e-02,
       8.26670825e-02, 8.26670825e-02, 3.36085923e-01, 3.36085923e-01,
       3.36085923e-01, 3.36085923e-01, 3.36085923e-01, 3.36085923e-01,
       7.48001237e-01, 7.48001237e-01, 7.51998753e-01, 7.51998753e-01])

#### Testing

In [256]:
vect = [-4.646928018967011e-01,2.770294629368069e-13,6.042253100720720e-13,-6.024516357133285e-13,-2.567187786718633e-13,1.437594192304828e-01,-2.763730452330614e-13,4.646928018967681e-01,6.783135520862164e-13,6.755585098225312e-13,-1.437594192304206e-01,2.565069368880155e-13,-6.036534951757788e-13,-6.777043000828106e-13,-6.255919748617283e-14,-2.875188384604817e-01,-6.292320178844996e-13,-5.606330583371935e-13,6.014298958703158e-13,-6.759194003610720e-13,-2.875188384604817e-01,-6.207393804663961e-14,-6.269833844031960e-13,5.590088393571927e-13,2.557458444682729e-13,-1.437594192304207e-01,6.287588334290853e-13,6.270036860946324e-13,4.000749631766127e-01,-2.383049928519640e-13,1.437594192304827e-01,-2.559215089660516e-13,5.600763473792744e-13,-5.580405389843103e-13,2.382296361695720e-13,-4.000749631765536e-01]
vect = np.array(vect)
vect

array([-4.64692802e-01,  2.77029463e-13,  6.04225310e-13, -6.02451636e-13,
       -2.56718779e-13,  1.43759419e-01, -2.76373045e-13,  4.64692802e-01,
        6.78313552e-13,  6.75558510e-13, -1.43759419e-01,  2.56506937e-13,
       -6.03653495e-13, -6.77704300e-13, -6.25591975e-14, -2.87518838e-01,
       -6.29232018e-13, -5.60633058e-13,  6.01429896e-13, -6.75919400e-13,
       -2.87518838e-01, -6.20739380e-14, -6.26983384e-13,  5.59008839e-13,
        2.55745844e-13, -1.43759419e-01,  6.28758833e-13,  6.27003686e-13,
        4.00074963e-01, -2.38304993e-13,  1.43759419e-01, -2.55921509e-13,
        5.60076347e-13, -5.58040539e-13,  2.38229636e-13, -4.00074963e-01])

In [167]:
np.sort(np.linalg.eigvals(rho_m(vect, rho_2_arrays).todense()))

array([6.39649654e-18+0.00000000e+00j, 2.27759896e-17-1.60872919e-17j,
       2.27759896e-17+1.60872919e-17j, 2.97340303e-17+0.00000000e+00j,
       1.05847838e-08+0.00000000e+00j, 1.05847839e-08+0.00000000e+00j,
       8.12469946e-02+0.00000000e+00j, 8.12469946e-02+0.00000000e+00j,
       8.12469946e-02+0.00000000e+00j, 8.12469946e-02+0.00000000e+00j,
       8.12469946e-02+0.00000000e+00j, 8.12469946e-02+0.00000000e+00j,
       8.26670825e-02+0.00000000e+00j, 8.26670825e-02+0.00000000e+00j,
       8.26670825e-02+0.00000000e+00j, 8.26670825e-02+0.00000000e+00j,
       8.26670825e-02+0.00000000e+00j, 8.26670825e-02+0.00000000e+00j,
       3.36085923e-01+0.00000000e+00j, 3.36085923e-01+0.00000000e+00j,
       3.36085923e-01+0.00000000e+00j, 3.36085923e-01+0.00000000e+00j,
       3.36085923e-01+0.00000000e+00j, 3.36085923e-01+0.00000000e+00j,
       7.48001237e-01+0.00000000e+00j, 7.48001237e-01+0.00000000e+00j,
       7.51998753e-01+0.00000000e+00j, 7.51998753e-01+0.00000000e+00j])